<a href="https://colab.research.google.com/github/Siglangf/Bigdata/blob/main/Project_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Kodelinje for å installere Spark på i Google Collab. Trengs bare å kjøre én gang. Forhåpentligvis ligger den inne etter det

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz 

In [ ]:
!tar xf spark-3.0.2-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
#For å legge til driven i notebooken så man kan få tilgang til filene som ligger i mappene
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl -L -o "/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.0-spark3.0-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.0-spark3.0-s_2.12/graphframes-0.8.0-spark3.0-s_2.12.jar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  237k  100  237k    0     0   786k      0 --:--:-- --:--:-- --:--:-- 7900k




Kodesnutten under må kjøres for å fortelle Python hvor den skal finne Spark filene

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.0.2-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config("spark.jars.packages", "graphframes:graphframes:0.7.0-spark2.4-s_2.11")\
        .getOrCreate()
#Legg til denne for å få GraphFrame pakken
spark.sparkContext.addPyFile(f'{os.environ["SPARK_HOME"]}/jars/graphframes-0.8.0-spark3.0-s_2.12.jar')

In [ ]:
#Legges inn som argument
DATA_PATH = "/content/drive/My Drive/Bigdata"

# Task 1

Task 1.1-4

In [ ]:
#Laster inn som en data
posts = spark.read.csv(path=f"{DATA_PATH}/posts.csv",sep=r'\t', header=True)
comments = spark.read.csv(path=f"{DATA_PATH}/comments.csv",sep=r'\t', header=True)
users = spark.read.csv(path=f"{DATA_PATH}/users.csv",sep=r'\t', header=True)
badges = spark.read.csv(path=f"{DATA_PATH}/badges.csv",sep=r'\t', header=True)

posts = posts.rdd
comments = comments.rdd
users = users.rdd
badges = badges.rdd

Task 1.5

In [ ]:
post_count = posts.count()
comments_count = comments.count()
users_count = users.count()
badges_count = badges.count()
print(post_count,comments_count,users_count,badges_count)

56217 58735 91616 105640


# Task 2


Task 2.1

In [ ]:
from pyspark.sql.functions import unbase64,base64
import base64

# Funksjon for å dekode base64-encoding
def decode_line(base64_message):
  base64_bytes = base64_message.encode('utf-8')
  message_bytes = base64.b64decode(base64_bytes)
  message = message_bytes.decode('utf-8')
  return message

# Dekoder kommentarer, finner lengden av dem, og lager nye rows med lengdene og ettall. 
# Deretter reduserer vi til en tupel med total lengde og antall kommentarer
comments_keyvalue = comments.map(lambda line: (len(decode_line(line['Text'])), 1)) \
              .reduce(lambda a, b : (a[0]+b[0], a[1]+b[1])) \

comments_avg = comments_length_count[0]/comments_length_count[1]

print("The average length of the comments is: " + str(comments_avg))

#Key/Value-pair bestående av PostTypeId og Body
post_id_body = posts.map(lambda line: (int(line['PostTypeId']), decode_line(line['Body'])))

# Gjør først om verdiene til tupler bestående av lengden på strengene og et ettall. Deretter summerer vi opp lengdene og countene 
# og finner snitt ved å dele lengde på count. 
avg_by_key = post_id_body.mapValues(lambda v: (len(v), 1)) \
            .reduceByKey(lambda a,b : ((a[0])+(b[0]), a[1]+b[1])) \
            .mapValues(lambda v: v[0]/v[1]) \
            .collectAsMap() 

print("The average length of question-posts is: " + str(avg_by_key[1]))
print("The average length of answer-posts is: " + str(avg_by_key[2]))

Task 2.2

In [ ]:
#Find the dates when the first and the last questions were asked. Also, find the display name of users who posted those questions
question_dates = posts.filter(lambda line : line['PostTypeId']=="1") \
                      .map(lambda line : (line['CreationDate'], line['OwnerUserId'])) \
                      .sortBy(lambda line : line[0], ascending = True)
first_date = question_dates.first()
#print("The first date is: " + str(first_date[0]))

question_dates = question_dates.sortBy(lambda line : line[0], ascending = False)
last_date = question_dates.first()
#print("The last date is: " + str(last_date[0]))

user_first_date = users.filter(lambda line : line['Id'] == first_date[1]) \
                       .map(lambda line: line['DisplayName'])

user_last_date = users.filter(lambda line : line['Id'] == last_date[1]) \
                       .map(lambda line: line['DisplayName'])

print("The user " + user_first_date.take(1)[0] + " asked the first question at " + str(first_date[0]))
print("The user " + user_last_date.take(1)[0] + " asked the last question at " + str(last_date[0]))

Task 2.3

In [ ]:
u = posts.map(lambda line: (line['OwnerUserId'],1))
u = u.filter(lambda line: line[0] != '-1')
u = u.reduceByKey(lambda x,y: int(x) + int(y))
counts = u.map(lambda line: line[1])
max_posts = counts.reduce(lambda x,y: max(x,y))
u = u.filter(lambda line: line[1]==max_posts)
u.take(1)

[('64377', 579)]

Task 2.4


In [ ]:
users_and_badges = badges.map(lambda badge: (badge['UserId'], 1))
num_badges = users_and_badges.reduceByKey(lambda x,y: int(x)+int(y))
above_3 = num_badges.filter(lambda x: x[1]>=3)
number_of_users = above_3.count()
print(f"{number_of_users} has above 3 badges")

9763 has above 3 badges


Task 2.5

In [ ]:
from operator import add
upvotes_user_map = users.map(lambda user: (user['Id'], user['UpVotes']))
#Drow None values and -1 user
upvotes_user_map = upvotes_user_map.filter(lambda user: not user[1]==None and not user[0]=='-1')


downvotes_user_map = users.map(lambda user: (user['Id'], user['DownVotes']))
#Drop none values and -1 user
downvotes_user_map = downvotes_user_map.filter(lambda user: not user[1]==None and not user[0]==-1)


#Compute mean of upvotes per user
upvotes = upvotes_user_map.map(lambda user: (int(user[1])))
agg_upvotes = upvotes.reduce(add)
mean_upvotes = agg_upvotes/upvotes_user_map.count()


#Compute mean of downvotes per user
downvotes = downvotes_user_map.map(lambda user: int(user[1]))
agg_downvotes = downvotes.reduce(add)
mean_downvotes = agg_downvotes/upvotes_user_map.count()


In [ ]:
from math import sqrt,pow
user_map = users.map(lambda user: (user['Id'],(user['UpVotes'],user['DownVotes'])))
user_map = user_map.filter(lambda user: not user[0]=='-1' and not user[1][0]==None and not user[1][1]==None)
variance = user_map.map(lambda user: (user[0], (int(user[1][0])-mean_upvotes)*(int(user[1][1])-mean_downvotes)))
variance_sum = variance.map(lambda user: user[1]).reduce(add)

upvote_squared = user_map.map(lambda user: (int(user[1][0])-mean_upvotes)**2)
downvote_squared = user_map.map(lambda user: (int(user[1][1])-mean_downvotes)**2)

upvote_squared_sum = upvote_squared.reduce(add)
downvote_squared_sum = downvote_squared.reduce(add)

pearson = variance_sum/(sqrt(upvote_squared_sum)*sqrt(downvote_squared_sum))

pearson

NameError: ignored

Task 2.6

In [ ]:
from math import log2
comment_map = comments.map(lambda comment: (comment['UserId'], 1))
comments_per_user = comment_map.reduceByKey(add)
total_comments = comment_map.count()
probability_per_user = comments_per_user.map(lambda user: (user[0],user[1]/total_comments))
entropy_user = probability_per_user.map(lambda prob: prob[1]*log2(prob[1]))
entropy = -entropy_user.reduce(add)
entropy

11.257354251781877

# Task 3


In [ ]:
posts = spark.read.csv(path=f"{DATA_PATH}/posts.csv",sep=r'\t', header=True)
comments = spark.read.csv(path=f"{DATA_PATH}/comments.csv",sep=r'\t', header=True)
users = spark.read.csv(path=f"{DATA_PATH}/users.csv",sep=r'\t', header=True)

posts = posts.rdd
comments = comments.rdd

Task 3.1

In [ ]:
#Make key/value pairs with postid and userIDs
posts_map = posts.filter(lambda line: line["OwnerUserId"]!="NULL").map(lambda post: (int(post['Id']), int(post['OwnerUserId'])))
comments_map = comments.filter(lambda line: line["UserId"]!="NULL").map(lambda comment: (int(comment['PostId']), int(comment['UserId'])))
#Joining on postid to get "PosterId and CommentID pairs"
joined =posts_map.join(comments_map)

In [ ]:
#Counting number of equal PosterId and CommentID pairs
counts = joined.map(lambda line: (line[1],1))
counts = counts.reduceByKey(lambda x,y: x+y)
#Creating graph tuples formated as (Posterid,Commenterid, Weight)
graph = counts.map(lambda line: (line[0][0],line[0][1],line[1]))

Task 3.2

In [ ]:
df = graph.toDF(["PosterId","CommenterId","Weight"])

In [ ]:
import pyspark.sql.functions as sf
top_ten_commenters = df.groupBy("CommenterId").agg(sf.sum("Weight").alias("Num_comments")).orderBy("Num_comments",ascending=False).limit(10)
top_ten_commenters = top_ten_commenters.select(top_ten_commenters.CommenterId.alias("Id"),"Num_comments")
top_ten_commenters.show()

+-----+------------+
|   Id|Num_comments|
+-----+------------+
|  836|        1263|
|  381|        1226|
|28175|         864|
|64377|         746|
|35644|         693|
|55122|         631|
|  924|         505|
|71442|         493|
|   21|         406|
|45264|         402|
+-----+------------+



Task 3.3

In [ ]:
#Finding userdi with most comments on posts
top_ten_posters = df.groupBy("PosterId").agg(sf.sum("Weight").alias("Num_comments")).orderBy("Num_comments",ascending=False).limit(10)
top_ten_posters = top_ten_posters.select(top_ten_posters.PosterId.alias("Id"),"Num_comments")

Task 3.4

In [ ]:
from pyspark.sql.types import IntegerType
#Converting Userids from string to Int
users = users.withColumn("Id", users["Id"].cast(IntegerType())).select("Id","DisplayName")

In [ ]:
#Joining with user table to find DisplayName
names = top_ten_posters.join(users, on="Id").select("DisplayName","Num_comments").orderBy("Num_comments",ascending=False)
names.show()

+--------------------+------------+
|         DisplayName|Num_comments|
+--------------------+------------+
|         Neil Slater|         875|
|               Erwan|         729|
|               Media|         703|
|             n1k31t4|         622|
|Has QUIT--Anony-M...|         502|
|            JahKnows|         362|
|               Leevo|         349|
|         David Masip|         318|
|          Noah Weber|         315|
|      Brian Spiering|         313|
+--------------------+------------+



Task 3.5

In [ ]:
csv = graph.map(lambda line: ",".join([str(l) for l in line]))
csv.saveAsTextFile(f"{DATA_PATH}/graph.csv")
